In [ ]:
import os 
import sys
import cv2
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from scipy.spatial.transform import Rotation as R

NOTEBOOKS_DIR = os.path.abspath("./")

%matplotlib inline
%reload_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


sys.path.append(f"{NOTEBOOKS_DIR}/..")

import sphere_snap.utils as snap_utils
import sphere_snap.sphere_coor_projections as sphere_proj
from sphere_snap.snap_config import SnapConfig, ImageProjectionType
from sphere_snap.sphere_snap import SphereSnap
import sphere_snap.reprojections as rpr


In [ ]:
def show_img(img):
    plt.figure(figsize=(13, 13))
    plt.imshow(img)
    plt.show()

def show_imgs(imgs, size = 14, nb_cols=2, title_txt= None, fontsize=10, imgs_text = None):
    """
    Display a gird of images 
    :param imgs: the list of images to be displayed 
    :param size: display size of a cell
    :param nb_cols: number of columns
    """

    row_size = int(np.ceil(len(imgs)/nb_cols))
    fig = plt.figure(figsize=(size,size))
    axes_pad = 0.1 if imgs_text is None else 0.5

    if title_txt is not None:
        fig.suptitle(title_txt, fontsize=fontsize)
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(row_size, nb_cols),  # creates 2x2 grid of axes
                     axes_pad=axes_pad,  # pad between axes in inch.
                     )

    for idx, data in enumerate(zip(grid, imgs)):
        ax, img  = data
        if imgs_text is not None:
             ax.set_title(imgs_text[idx], fontdict={'fontsize': 15, 'fontweight': 'medium'}, loc='center', color = "k")
        ax.imshow(img)
    plt.show()
    
def rot(yaw, pitch):
    return R.from_euler("yxz",[yaw,-pitch,0], degrees=True).as_quat()


In [ ]:
equi_img_path =f"{NOTEBOOKS_DIR}/../data/equi_test_img.jpg"
equi_img = plt.imread(equi_img_path)
show_img(equi_img)

### Snap to perspective from equirectangular

In [ ]:
snap_config = SnapConfig( [0,0,0,1], (1400,1400),(120,120), equi_img.shape[:2], source_img_type=ImageProjectionType.EQUI)
snap_test = SphereSnap(snap_config)
persp_img = snap_test.snap_to_perspective(equi_img)
show_img(persp_img)

### Reproject equirectangular into 6 planar images of 90 degrees FoV (Cubemap)

In [ ]:
cube_faces_imgs = rpr.equi2cubemap(equi_img)
show_imgs(cube_faces_imgs, nb_cols=6)

### Reproject a planar image into equirectangular

In [ ]:
reconstructed_equi = rpr.cubemap2equi(cube_faces_imgs[::-1])
show_img(reconstructed_equi)


In [ ]:
show_imgs(rpr.equi2fisheye(equi_img))

### Reproject fromm equirectangular to Birds-Eye-View

In [ ]:
tv_img = rpr.equi2tv(equi_img, np.identity(3), (1000,1000), (90,90), 10, 2)
show_img(tv_img)

### Reproject a planar image into fisheye 180

In [ ]:
reconstructed_fisheye = SphereSnap.merge_multiple_snaps((1000,1000), 
                                                     [snap_test], # snap object specifies destination position
                                                     [persp_img], # snap image contains planar image pixels
                                                     target_type=ImageProjectionType.FISHEYE_180, # destination image type
                                                     merge_method="max")
show_img(reconstructed_fisheye)


### Snap to perspective from fisheye 180

In [ ]:
snap_config = SnapConfig( rot(25,1), (1400,1400),(100,100), reconstructed_fisheye.shape[:2], source_img_type=ImageProjectionType.FISHEYE_180)
snap_test = SphereSnap(snap_config)
persp_img = snap_test.snap_to_perspective(reconstructed_fisheye)
show_img(persp_img)

### Reproject fisheye 180 to equirectangular

In [ ]:
fisheye180_img_path =f"{NOTEBOOKS_DIR}/../data/fisheye180_test_img.jpg"
fisheye180_img = plt.imread(fisheye180_img_path)
fisheye180_img = cv2.resize(fisheye180_img, (1000,1000))
show_img(fisheye180_img)

In [ ]:
snap_configs = [SnapConfig( rot(yaw,pitch), (800,800),(110,110), fisheye180_img.shape[:2], source_img_type=ImageProjectionType.FISHEYE_180) 
                    for yaw,pitch in [[45,0],[-45,0],[0,45],[0,-45]]]
snaps = [SphereSnap(c) for c in snap_configs]
snap_imgs = [snap.snap_to_perspective(fisheye180_img) for snap in snaps]
show_imgs(snap_imgs, nb_cols=7)

In [ ]:
reconstructed_equi = SphereSnap.merge_multiple_snaps((1000,2000), 
                                                     snaps, # snap object specifies destination position
                                                     snap_imgs, # snap image contains planar image pixels
                                                     target_type=ImageProjectionType.EQUI, # destination image type
                                                     merge_method="max")
show_img(reconstructed_equi)


In [ ]:
show_img(rpr.fisheye2equi(fisheye180_img,cv2.flip(fisheye180_img,1)))